Homework 5
By: Hana McVicker
Github Link: https://github.com/hanamcvicker/EE399

In [25]:
import numpy as np
from scipy import integrate
import torch
import torch.nn as nn
import torch.optim as optim

For the Lorenz equations (code given out previously in class emails), consider the following.
1. Train a NN (FFNN) to advance the solution from t to t + ∆t for ρ = 10, 28 and 40. Now see how well
your NN (FFNN) works for future state prediction for ρ = 17 and ρ = 35.

In [26]:
def lorenz(rho):
        # Initialize parameters
    dt = 0.01
    T = 8
    t = np.arange(0,T+dt,dt)
    beta = 8/3
    sigma = 10

    # Initialize input and output arrays for rho value
    nn_input = np.zeros((100 * (len(t) - 1), 3))
    nn_output = np.zeros_like(nn_input)

    # Define Lorenz system
    def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
        x, y, z = x_y_z
        return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]
    
    # Solve Lorenz system for rho value
    np.random.seed(123)
    x0 = -15 + 30 * np.random.random((100, 3))

    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t)
                    for x0_j in x0])

    for j in range(100):
        nn_input[j*(len(t)-1):(j+1)*(len(t)-1),:] = x_t[j,:-1,:]
        nn_output[j*(len(t)-1):(j+1)*(len(t)-1),:] = x_t[j,1:,:]
    
    return nn_input, nn_output


In [31]:
# Training

# Define activation functions
def logsig(x):
    return 1 / (1 + torch.exp(-x))

def radbas(x):
    return torch.exp(-torch.pow(x, 2))

def purelin(x):
    return x

# Define the model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(in_features=3, out_features=10)
        self.fc2 = nn.Linear(in_features=10, out_features=10)
        self.fc3 = nn.Linear(in_features=10, out_features=3)

    def forward(self, x):
        x = logsig(self.fc1(x))
        x = radbas(self.fc2(x))
        x = purelin(self.fc3(x))
        return x

# Create model instance
model = MyModel()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Generate training data
rho_train = [10, 28, 40]
nn_input = np.zeros((0, 3))
nn_output = np.zeros_like(nn_input)

for i, rho in enumerate(rho_train):
        nn_input_rho, nn_output_rho = lorenz(rho)
        nn_input = np.concatenate((nn_input, nn_input_rho))
        nn_output = np.concatenate((nn_output, nn_output_rho))  
        
# Convert numpy arrays to PyTorch tensors
nn_input_torch = torch.from_numpy(nn_input).float()
nn_output_torch = torch.from_numpy(nn_output).float()

# Train the model
for epoch in range(30):
    optimizer.zero_grad()
    outputs = model(nn_input_torch)
    loss = criterion(outputs, nn_output_torch)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, loss={loss.item():.4f}")

Epoch 1, loss=292.5014
Epoch 2, loss=270.6131
Epoch 3, loss=251.2502
Epoch 4, loss=233.5440
Epoch 5, loss=217.3198
Epoch 6, loss=202.8074
Epoch 7, loss=190.1171
Epoch 8, loss=179.1136
Epoch 9, loss=169.6170
Epoch 10, loss=161.4575
Epoch 11, loss=154.4547
Epoch 12, loss=148.4359
Epoch 13, loss=143.2541
Epoch 14, loss=138.7868
Epoch 15, loss=134.9302
Epoch 16, loss=131.5955
Epoch 17, loss=128.7056
Epoch 18, loss=126.1938
Epoch 19, loss=124.0021
Epoch 20, loss=122.0801
Epoch 21, loss=120.3844
Epoch 22, loss=118.8774
Epoch 23, loss=117.5259
Epoch 24, loss=116.2994
Epoch 25, loss=115.1678
Epoch 26, loss=114.1008
Epoch 27, loss=113.0667
Epoch 28, loss=112.0376
Epoch 29, loss=110.9986
Epoch 30, loss=109.9324


In [32]:
# Testing FFNN for future state prediction for ρ = 17 and ρ = 35.
test_values = [17, 35]

for rho in test_values:
    ffnn_test_input, ffnn_test_output = lorenz(rho)
    ffnn_test_input = torch.from_numpy(ffnn_test_input).float()
    ffnn_test_output = torch.from_numpy(ffnn_test_output).float()
    ffnn_output_pred = model(ffnn_test_input)
    loss = criterion(ffnn_output_pred, ffnn_test_output)
    print('Loss for rho = ', rho, ': ', loss.item())

Loss for rho =  17 :  42.18996810913086
Loss for rho =  35 :  135.71096801757812


2. Compare LSTM, RNN and Echo State Networks for forecasting the dynamics.

In [34]:
# Create LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size=3, hidden_layer_size=10, output_size=3):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_layer_size)
        c0 = torch.zeros(1, x.size(0), self.hidden_layer_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.linear(out[:, -1, :])
        return out

In [35]:
# Create model instance
model3 = LSTM()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model3.parameters(), lr=0.01, momentum=0.9)

# Reshape the input data for LSTM
nn_input_lstm = nn_input.reshape(-1, 1, 3)
nn_input_lstm = torch.from_numpy(nn_input_lstm).float()


# Train the model
for epoch in range(30):
    optimizer.zero_grad()
    outputs = model3(nn_input_lstm)
    loss = criterion(outputs, nn_output_torch)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, loss={loss.item():.4f}")

Epoch 1, loss=296.1241
Epoch 2, loss=287.8757
Epoch 3, loss=280.3617
Epoch 4, loss=269.8288
Epoch 5, loss=253.8643
Epoch 6, loss=233.2905
Epoch 7, loss=212.0084
Epoch 8, loss=190.2160
Epoch 9, loss=169.1255
Epoch 10, loss=149.5753
Epoch 11, loss=132.7411
Epoch 12, loss=119.3574
Epoch 13, loss=109.1402
Epoch 14, loss=102.3229
Epoch 15, loss=98.4028
Epoch 16, loss=96.2859
Epoch 17, loss=94.5471
Epoch 18, loss=93.0146
Epoch 19, loss=94.9779
Epoch 20, loss=93.3743
Epoch 21, loss=94.3646
Epoch 22, loss=94.0265
Epoch 23, loss=92.4241
Epoch 24, loss=84.0550
Epoch 25, loss=93.3060
Epoch 26, loss=95.3480
Epoch 27, loss=87.8754
Epoch 28, loss=72.5454
Epoch 29, loss=95.0573
Epoch 30, loss=103.1637


In [36]:
# Test the LSTM Model for ρ = 17 and ρ = 35
for rho in test_values:
    lstm_test_input, lstm_test_output = lorenz(rho)
    lstm_test_input = lstm_test_input.reshape(-1, 1, 3)
    lstm_test_input = torch.from_numpy(lstm_test_input).float()
    lstm_test_output = torch.from_numpy(lstm_test_output).float()
    lstm_output_pred = model3(lstm_test_input)
    loss = criterion(lstm_output_pred, lstm_test_output)
    print('Loss for rho = ', rho, ': ', loss.item())

Loss for rho =  17 :  14.785507202148438
Loss for rho =  35 :  102.20310974121094


#### RNN

In [39]:
# RNN 
class RNN(nn.Module):
    def __init__(self, input_size=3, hidden_layer_size=10, output_size=3):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.rnn = nn.RNN(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_layer_size)
        out, _ = self.rnn(x, h0)
        out = self.linear(out[:, -1, :])
        return out

In [40]:
# Create model instance
model4 = RNN()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model4.parameters(), lr=0.01, momentum=0.9)

# Reshape the input data for RNN
nn_input_rnn = nn_input.reshape(-1, 1, 3)
nn_input_rnn = torch.from_numpy(nn_input_rnn).float()

# Train the model
for epoch in range(30):
    optimizer.zero_grad()
    outputs = model4(nn_input_rnn)
    loss = criterion(outputs, nn_output_torch)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, loss={loss.item():.4f}")

Epoch 1, loss=304.0966
Epoch 2, loss=276.8466
Epoch 3, loss=241.2286
Epoch 4, loss=199.3394
Epoch 5, loss=156.8338
Epoch 6, loss=124.2359
Epoch 7, loss=104.8441
Epoch 8, loss=96.9755
Epoch 9, loss=86.7712
Epoch 10, loss=115.9577
Epoch 11, loss=105.1501
Epoch 12, loss=97.6501
Epoch 13, loss=85.2472
Epoch 14, loss=85.5213
Epoch 15, loss=85.6785
Epoch 16, loss=84.2939
Epoch 17, loss=83.1829
Epoch 18, loss=82.0210
Epoch 19, loss=80.4448
Epoch 20, loss=78.4878
Epoch 21, loss=76.4563
Epoch 22, loss=74.6939
Epoch 23, loss=73.2980
Epoch 24, loss=72.0764
Epoch 25, loss=70.1151
Epoch 26, loss=67.1871
Epoch 27, loss=69.0773
Epoch 28, loss=67.8966
Epoch 29, loss=66.6526
Epoch 30, loss=68.6693


In [41]:
# Test the RNN Model for ρ = 17 and ρ = 35
for rho in test_values:
    rnn_test_input, rnn_test_output = lorenz(rho)
    rnn_test_input = rnn_test_input.reshape(-1, 1, 3)
    rnn_test_input = torch.from_numpy(rnn_test_input).float()
    rnn_test_output = torch.from_numpy(rnn_test_output).float()
    rnn_output_pred = model4(rnn_test_input)
    loss = criterion(rnn_output_pred, rnn_test_output)
    print('Loss for rho = ', rho, ': ', loss.item())


Loss for rho =  17 :  31.507925033569336
Loss for rho =  35 :  67.714599609375


#### Echo State Network (ESN)

In [42]:
# Echo State Network for Lorenz System
class Reservoir(nn.Module):
  def __init__(self, hidden_dim, connectivity):
    super().__init__()
    
    self.Wx = self.sparse_matrix(hidden_dim, connectivity)
    self.Wh = self.sparse_matrix(hidden_dim, connectivity)
    self.Uh = self.sparse_matrix(hidden_dim, connectivity)
    self.act = nn.Tanh()

  def sparse_matrix(self, m, p):
    mask_distribution = torch.distributions.Bernoulli(p)
    S = torch.randn((m, m))
    mask = mask_distribution.sample(S.shape)
    S = (S*mask).to_sparse()
    return S

  def forward(self, x, h):
    h = self.act(torch.sparse.mm(self.Uh, h.T).T +
                 torch.sparse.mm(self.Wh, x.T).T)
    y = self.act(torch.sparse.mm(self.Wx, h.T).T)

    return y, h
     
class EchoState(nn.Module):
  def __init__(self, in_dim, out_dim, reservoir_dim, connectivity):
    super().__init__()

    self.reservoir_dim = reservoir_dim
    self.input_to_reservoir = nn.Linear(in_dim, reservoir_dim)
    self.input_to_reservoir.requires_grad_(False)

    self.reservoir = Reservoir(reservoir_dim, connectivity)
    self.readout = nn.Linear(reservoir_dim, out_dim)
  
  def forward(self, x):
    reservoir_in = self.input_to_reservoir(x)
    h = torch.ones(x.size(0), self.reservoir_dim)
    reservoirs = []
    for i in range(x.size(1)):
      out, h = self.reservoir(reservoir_in[:, i, :], h)
      reservoirs.append(out.unsqueeze(1))
    reservoirs = torch.cat(reservoirs, dim=1)
    outputs = self.readout(reservoirs)
    return outputs

In [43]:
# Create model instance
model5 = EchoState(3, 3, 100, 0.1)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model5.parameters(), lr=0.01, momentum=0.9)

# Train the model
for epoch in range(30):
    optimizer.zero_grad()
    outputs = model5(nn_input_torch.view(1, -1, 3))
    loss = criterion(outputs.squeeze(), nn_output_torch)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, loss={loss.item():.4f}")

Epoch 1, loss=290.8023
Epoch 2, loss=191.0247
Epoch 3, loss=93.6039
Epoch 4, loss=76.8121
Epoch 5, loss=124.5946
Epoch 6, loss=162.4149
Epoch 7, loss=143.1104
Epoch 8, loss=88.1637
Epoch 9, loss=52.3589
Epoch 10, loss=61.9440
Epoch 11, loss=93.0898
Epoch 12, loss=104.6197
Epoch 13, loss=82.2997
Epoch 14, loss=48.4096
Epoch 15, loss=33.3772
Epoch 16, loss=44.4759
Epoch 17, loss=62.9991
Epoch 18, loss=67.3426
Epoch 19, loss=54.3991
Epoch 20, loss=38.6192
Epoch 21, loss=34.0801
Epoch 22, loss=40.6024
Epoch 23, loss=46.8435
Epoch 24, loss=43.9613
Epoch 25, loss=34.1769
Epoch 26, loss=26.6329
Epoch 27, loss=27.0264
Epoch 28, loss=32.4960
Epoch 29, loss=35.8967
Epoch 30, loss=33.5237


In [44]:
# # Test the ESN Model for ρ = 17 and ρ = 35
for rho in test_values:
    esn_test_input, esn_test_output = lorenz(rho)
    esn_test_input = torch.from_numpy(esn_test_input).float()
    esn_test_output = torch.from_numpy(esn_test_output).float()
    esn_output_pred = model5(esn_test_input.view(1, -1, 3))
    loss = criterion(esn_output_pred.squeeze(), esn_test_output)
    print('Loss for rho = ', rho, ': ', loss.item())



Loss for rho =  17 :  42.07099914550781
Loss for rho =  35 :  27.34931755065918
